# Annotate significant GWAS results with gnomAD

<table align="left">

  <td>
    <a href="https://github.com/DataBiosphere/terra-axon-examples/blob/main/dataproc/annotate_significant_gwas_results_with_gnomad.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/DataBiosphere/terra-axon-examples/main/dataproc/annotate_significant_gwas_results_with_gnomad.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in a Terra notebook instance
    </a>
  </td>                                                                                               
</table>

In this notebook, we use [Hail](https://hail.is/) to annotate the significant GWAS results with [gnomAD](https://gnomad.broadinstitute.org/).

GWAS results from *"Whole genome sequence analysis of blood lipid levels in >66,000 individuals. [Selvaraj et al 2021](https://www.biorxiv.org/content/10.1101/2021.10.11.463514v1.supplementary-material)"* are used. There are only 338 significant LDL results to annotate, so it might be faster to annotate using a tool other than Hail, but this notebook is meant to be illustrative.

The gnomAD table is enormous and this notebook makes use of [filter_intervals](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.filter_intervals) to remove irrelevant gnomAD data partitions from the analysis.
* If you want this notebook to run quickly (~5 min), be sure to set `INTERVALS_TO_EXAMINE` to a small region of the genome such as APOE.
* If you want this notebook to run for a while (~30 minutes), set `INTERVALS_TO_EXAMINE` to include all autosomes.

If you want to run this notebook in batch mode as a script, in the terminal run `jupyter nbconvert --to script annotate_significant_gwas_results_with_gnomad.ipynb`.

# Setup 

In [ ]:
from datetime import datetime
import hail as hl
import os
import pandas as pd
from plotnine import *
import subprocess
import time

## Define constants

In [ ]:
# Papermill parameters. See https://papermill.readthedocs.io/en/latest/usage-parameterize.html

#---[ Inputs ]---
# The gnomAD v3.1.2 data set contains 76,156 whole genomes (and no exomes), all mapped to the GRCh38 reference sequence.
# See also https://gnomad.broadinstitute.org/downloads
GNOMAD_TAB = 'gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht'
# GRCh38 GWAS results from https://www.biorxiv.org/content/10.1101/2021.10.11.463514v1.supplementary-material
SELVARAJ_LIPIDS_GWAS_RESULTS = 'https://www.biorxiv.org/content/biorxiv/early/2021/10/12/2021.10.11.463514/DC1/embed/media-1.xlsx?download=true'

# To run at scale, use these intervals for all autosomes.
#INTERVALS_TO_EXAMINE = ['chr1-chr22'] 
# To perform a quick test, use these intervals for APOE
# https://genome.ucsc.edu/cgi-bin/hgTracks?db=hg38&lastVirtModeType=default&lastVirtModeExtraState=&virtModeType=default&virtMode=0&nonVirtPosition=&position=chr19%3A44905796%2D44909393&hgsid=1411550209_Ip7W0XTmGpvEfKONETLScNT3MOVv
INTERVALS_TO_EXAMINE = ['chr19:44905796-44909393']

#---[ Outputs ]---
OUTPUT_BUCKET = os.getenv('WORKSPACE_BUCKET')

In [ ]:
#---[ Form output path from parameter values ]---
if not OUTPUT_BUCKET:
    # This notebook is not running in app.terra.bio or the All of Us Researcher Workbench.
    raise ValueError('Specify the value for parameter OUTPUT_BUCKET.')

INTERVALS_TO_EXAMINE_NAME = '_'.join(INTERVALS_TO_EXAMINE).replace(':', 'range')

# Create a timestamp for a folder of results generated today.
DATESTAMP = time.strftime('%Y%m%d')
TIMESTAMP = time.strftime('%Y%m%d_%H%M%S')
WORK_DIR = os.getcwd()

RESULTS_DIR = f'{OUTPUT_BUCKET}/data/results/{DATESTAMP}/'
ANNOTATED_LIPIDS_GWAS_RESULTS = f'significant_ldl_gwas_results_tbl_gnomad_annotated-{INTERVALS_TO_EXAMINE_NAME}.tsv'

## Start Hail 

In [ ]:
hl.init(default_reference='GRCh38')

In [ ]:
start_all = datetime.now()
print(start_all)

# Read the gnomAD variant annotation table

In [ ]:
gnomad = hl.read_table(GNOMAD_TAB)

In [ ]:
gnomad.count()

In [ ]:
gnomad.describe()

# Or use this command to view the interactive version of the description.
#gnomad.describe(widget=True)

# Read the lipids GWAS results table

In [ ]:
selvaraj_gwas_results_df = pd.read_excel(SELVARAJ_LIPIDS_GWAS_RESULTS,
                                       sheet_name = 'Supplementary Table 3',
                                       skiprows = 3
                                      )

selvaraj_gwas_results_df.shape

## Reformat the data for computation

Note that this particular sheet is well formated for reading, but less so for computation. Results for the various lipids are found within particular row ranges within the spreadsheet.

In [ ]:
# This is the header row for LDL results.
selvaraj_gwas_results_df.iloc[357,]

In [ ]:
# This is the header row for total cholesterol results.
selvaraj_gwas_results_df.iloc[697,]

In [ ]:
# Extract the the subset of rows within the spreadsheet holding just the LDL GWAS results.
ldl_gwas_results_df = selvaraj_gwas_results_df.iloc[359:697,]

# Fix the data types on the columns.
ldl_gwas_results_df = ldl_gwas_results_df.astype({
    'CHR': 'int64',
    'POS': 'int64',
    'BETA': 'float64',
    'SE': 'float64',
    'p.value': 'float64',
    'MAF': 'float64'
})

# Add a categorical variable for chromosome.
ldl_gwas_results_df['CHROM'] = pd.Categorical('chr' + ldl_gwas_results_df.CHR.astype('string'),
                                              ordered=True,
                                              categories=[f'chr{i}' for i in range(1, 23)])

# Sort the dataframe.
ldl_gwas_results_df.sort_values(by=['CHR', 'POS'], inplace=True)

In [ ]:
ldl_gwas_results_df.head()

In [ ]:
ldl_gwas_results_df.tail()

In [ ]:
(ggplot(ldl_gwas_results_df, aes(x = 'CHROM')) +
geom_bar() +
theme_bw()+
theme(figure_size=(16, 8)) +
labs(title = 'Distribution of significant GWAS results per chromosome'))

## Create a Hail table of the GWAS results

In [ ]:
# Convert the pandas dataframe to a Hail table.
ldl_gwas_results_tbl = hl.Table.from_pandas(
    ldl_gwas_results_df[['CHROM', 'POS', 'Allele1', 'Allele2', 'BETA', 'SE', 'p.value', 'MAF']])

In [ ]:
ldl_gwas_results_tbl.count()

### Create the `locus` and `alleles` fields need to join with Hail matrix tables

In [ ]:
ldl_gwas_results_tbl = ldl_gwas_results_tbl.annotate(locus=hl.locus(ldl_gwas_results_tbl.CHROM, ldl_gwas_results_tbl.POS))

In [ ]:
ldl_gwas_results_tbl = ldl_gwas_results_tbl.annotate(alleles=hl.array([ldl_gwas_results_tbl.Allele1,
                                                                       ldl_gwas_results_tbl.Allele2]))

In [ ]:
ldl_gwas_results_tbl = ldl_gwas_results_tbl.key_by(ldl_gwas_results_tbl.locus,
                                                   ldl_gwas_results_tbl.alleles)

In [ ]:
ldl_gwas_results_tbl.describe()

# Or use this command to view the interactive version of the description.
#ldl_gwas_results_tbl.describe(widget=True)

# Try to improve performance

## Filter gnomAD to include only our genomic intervals of interest

In [ ]:
gnomad.n_partitions()

In [ ]:
# First filter using the manually defined intervals.
gnomad = hl.filter_intervals(
    gnomad,
    [hl.parse_locus_interval(x) for x in INTERVALS_TO_EXAMINE],
    keep=True)

In [ ]:
gnomad.n_partitions()

In [ ]:
# Compute the genomic intervals forming the outer bounds around our GWAS results. We will use this to read fewer partitions from gnomAD.
relevant_intervals_df = ldl_gwas_results_df.groupby('CHR').agg(min_pos=('POS', 'min'), max_pos=('POS', max))
relevant_intervals = relevant_intervals_df.apply(lambda x: f'chr{x.name}:{x["min_pos"]}-{x["max_pos"]+1}', axis=1).tolist()

relevant_intervals

In [ ]:
# Now filter further to only include data overlapping the GWAS results.
gnomad = hl.filter_intervals(
    gnomad,
    [hl.parse_locus_interval(x) for x in relevant_intervals],
    keep=True)

In [ ]:
gnomad.n_partitions()

## Repartition to (try to) improve parallelism

In [ ]:
ldl_gwas_results_tbl.n_partitions()

In [ ]:
ldl_gwas_results_tbl = ldl_gwas_results_tbl.repartition(ldl_gwas_results_tbl.count())

In [ ]:
ldl_gwas_results_tbl.n_partitions()

# Annotate significant lipids GWAS results with gnomAD

## Write annotated lipids GWAS results to TSV

In [ ]:
start = datetime.now()
print(start)

In [ ]:
annotated_ldl_gwas_results_tbl = ldl_gwas_results_tbl.join(gnomad, how='left')

In [ ]:
annotated_ldl_gwas_results_tbl.describe()

# Or use this command to view the interactive version of the description.
#annotated_ldl_gwas_results_tbl.describe(widget=True)

In [ ]:
annotated_ldl_gwas_results_tbl.export(os.path.join(RESULTS_DIR, ANNOTATED_LIPIDS_GWAS_RESULTS))

In [ ]:
end = datetime.now()
print(end)
print(end - start)

## Examine the ancestry distribution of the GWAS results

In [ ]:
gnomad_annotated_ldl_gwas_results_tbl = hl.import_table(os.path.join(RESULTS_DIR, ANNOTATED_LIPIDS_GWAS_RESULTS))

In [ ]:
gnomad_annotated_ldl_gwas_results_tbl.describe()

# Or use this command to view the interactive version of the description.
#gnomad_annotated_ldl_gwas_results_tbl.describe(widget=True)

In [ ]:
gnomad_annotated_ldl_gwas_results_tbl.filter(hl.is_defined(gnomad_annotated_ldl_gwas_results_tbl.popmax)).popmax.show()

In [ ]:
gnomad_annotated_ldl_gwas_results_tbl.aggregate(hl.agg.counter(hl.parse_json(
    gnomad_annotated_ldl_gwas_results_tbl.popmax,
    dtype='struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32, pop: str, faf95: float64}').pop))

# Provenance

In [ ]:
end_all = datetime.now()
print(end_all)
print(end_all - start_all)

In [ ]:
process_output = subprocess.run(['pip3', 'freeze'], capture_output=True, text=True)
print(process_output.stdout)

---
Copyright 2023 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style   
license that can be found in the LICENSE file or at   
https://developers.google.com/open-source/licenses/bsd